<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Natural%20Language%20Processing/Introduction_to_NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in Tensorflow

In [76]:
!nvidia-smi

Mon Feb 20 18:15:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    30W /  70W |   1186MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [77]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-20 18:15:18--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-20 18:15:18 (114 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [78]:
from helper_functions import plot_loss_curves,unzip_data,create_tensorboard_callback,compare_historys

## Getting a text Dataset

In [79]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-20 18:15:19--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.147.128, 142.250.125.128, 142.250.136.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.147.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-02-20 18:15:19 (136 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [80]:
unzip_data("nlp_getting_started.zip")

## Visualizing Dataset

In [81]:
import pandas as pd
train_dataset = pd.read_csv("/content/train.csv")

In [82]:
train_dataset

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [83]:
train_dataset_shuffled = train_dataset.sample(frac = 1,random_state = 42)


In [84]:
train_dataset_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [85]:
test_data  = pd.read_csv("/content/test.csv")

In [86]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [87]:
train_dataset.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [88]:
import random
random_index = random.randint(0,len(train_dataset)-5)
for row in train_dataset_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(text,target)

@violentfeminazi I guess that's ok for Armenians since we've spent most of our history getting annihilated 1
@CoreyAshe Did that look broken or bleeding? 0
Literally trapped in my room Cuz my bathroom being remodeled. The only exit is through a window 1
@NBCNews Yea bombing #pearlharbor not so good of an idea! 1
Ebola: Alabama Home Quarantined Over Possible Ebola Case.. Related Articles: http://t.co/BiigD1LEq3 1


## Split Data into Training and Validation Sets

In [89]:
from sklearn.model_selection import train_test_split

In [90]:
train_sentences,val_sentences,train_labels,val_labels = train_test_split(
    train_dataset_shuffled['text'].to_numpy(),
    train_dataset_shuffled['target'].to_numpy(),
    test_size = 0.1,
    random_state = 42
    )

In [91]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

## Tokenization and Embedding

### Text Vectorization (Tokenization)

In [92]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens = 10000,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None,
    pad_to_max_tokens = True,
  )


In [93]:
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

14.901036345059115

### Text Vecotrizer Variables

In [94]:
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)

In [95]:
text_vectorizer.adapt(train_sentences)

In [96]:
sample_sentence = "Flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [97]:
random_sentences = random.choice(train_sentences)
text_vectorizer([random_sentences])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 963,   58,  544, 1784,   85,  757,  465,    4,  941,  928,  777,
         232,    1,    0,    0]])>

In [98]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
top_5_words

['', '[UNK]', 'the', 'a', 'in']

## Embedding Layer

In [127]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
      output_dim = 128,
      input_length = max_length
                                   
)
embedding

In [100]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
embedded_sentence = embedding(text_vectorizer(random_sentence))
print(embedded_sentence)

@ClassyColkett Thorgan Hazard made his move permanent go Gladbach this summer lmao
tf.Tensor(
[[-0.03311805 -0.03422745  0.01141268 ...  0.01471348  0.03485749
   0.02838686]
 [-0.04218352  0.03129831 -0.02548726 ...  0.00367896  0.00246843
   0.0151406 ]
 [-0.02379868 -0.00181819 -0.0369621  ... -0.00180038  0.04635927
  -0.02537736]
 ...
 [-0.04770672  0.02680076 -0.01654663 ... -0.04818648  0.01237912
   0.01181265]
 [-0.04770672  0.02680076 -0.01654663 ... -0.04818648  0.01237912
   0.01181265]
 [-0.04770672  0.02680076 -0.01654663 ... -0.04818648  0.01237912
   0.01181265]], shape=(15, 128), dtype=float32)


## Model 0 : Naive Bayes (Baseline)

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convert words to numbers,
    ("clf",MultinomialNB()), #model the text,
])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [102]:
base_line_score = model_0.score(val_sentences,val_labels)

In [103]:
base_line_score

0.7926509186351706

In [104]:
base_line_predict = model_0.predict(val_sentences)
base_line_predict[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating functions to compare model experiments

In [105]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = "weighted")
  model_results = {
      "accuracy":model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":model_f1
  }
  return model_results

In [106]:
base_line_results = calculate_results(val_labels,base_line_predict)
base_line_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: Simple Dense Model

In [107]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [108]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")

In [109]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [110]:
model_1.compile(loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy'])

In [111]:
model_1_history = model_1.fit(
    x = train_sentences,y = train_labels,
    epochs = 5,
    validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_1_dense")]
    )

Saving TensorBoard log files to: model_logs/model_1_dense/20230220-181520
Epoch 1/5
215/215 [==============================] - 9s 37ms/step - loss: 0.6344 - accuracy: 0.6533 - val_loss: 0.5721 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4710 - accuracy: 0.8205 - val_loss: 0.4798 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3454 - accuracy: 0.8724 - val_loss: 0.4588 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.2607 - accuracy: 0.9066 - val_loss: 0.4634 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2005 - accuracy: 0.9345 - val_loss: 0.4716 - val_accuracy: 0.7887


In [112]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4716 - accuracy: 0.7887


[0.47156822681427, 0.7887139320373535]

In [113]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [114]:
model_1_pred_probs[10]

array([0.2201562], dtype=float32)

In [115]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [116]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [117]:
model_1_results = calculate_results(y_true = val_labels,y_pred = model_1_preds)
model_1_results

{'accuracy': 78.87139107611549,
 'precision': 0.7926581572076621,
 'recall': 0.7887139107611548,
 'f1': 0.7860944810879305}

## Visualizing Learned Embeddings

In [118]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [119]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [128]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]

In [129]:
embed_weights.shape

(10000, 128)

In [130]:
embed_weights

array([[-0.12979561, -0.03062526, -0.10476004, ..., -0.0611692 ,
        -0.03905344, -0.03014161],
       [-0.09555553, -0.04093374, -0.02216236, ..., -0.04311356,
        -0.05650194, -0.04710473],
       [-0.05348639, -0.04480572, -0.08218617, ..., -0.05637785,
        -0.0311494 , -0.02752293],
       ...,
       [ 0.02418255, -0.02899865, -0.04862274, ...,  0.04314418,
        -0.04538028,  0.02775973],
       [-0.0572253 ,  0.04743423,  0.00484018, ...,  0.00912398,
        -0.02672455,  0.04830042],
       [-0.04817164,  0.06603009, -0.0948725 , ...,  0.04523518,
         0.05075543,  0.15733586]], dtype=float32)

## Recurrent Neural Network

### Model 2: LSTM

In [131]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64,return_sequences = True)(x) # When we are stacking we need return_sequences = True
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64,activation = "relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [132]:
model_2.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [133]:
model_2_history = model_2.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_2_lstm")]
            )

Saving TensorBoard log files to: model_logs/model_2_lstm/20230220-181659
Epoch 1/5
215/215 [==============================] - 14s 42ms/step - loss: 0.5083 - accuracy: 0.7495 - val_loss: 0.4746 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3243 - accuracy: 0.8698 - val_loss: 0.5113 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2183 - accuracy: 0.9203 - val_loss: 0.5571 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.1650 - accuracy: 0.9406 - val_loss: 0.6844 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1204 - accuracy: 0.9539 - val_loss: 0.8520 - val_accuracy: 0.7612


In [134]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[0.01119321],
       [0.8440391 ],
       [0.9999256 ],
       [0.02494149],
       [0.0057727 ],
       [0.99974173],
       [0.2866949 ],
       [0.99998105],
       [0.9999591 ],
       [0.22453089]], dtype=float32)

In [135]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [136]:
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [137]:
calculate_results(val_labels,model_2_preds)

{'accuracy': 76.11548556430446,
 'precision': 0.7639062094795714,
 'recall': 0.7611548556430446,
 'f1': 0.758270611394164}

### Model-3 GRU Powered RNN

In [138]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#If returns sequences and stackings are true
# x = layers.LSTM(64,return_sequences = True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64,activation = "relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [139]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_2 (GRU)                 (None, 64)                37248     
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [140]:
model_3.compile(loss = "binary_crossentropy",metrics = ['accuracy'],optimizer = "adam")

In [141]:
model_3_history = model_3.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_3_gru")]
            )

Saving TensorBoard log files to: model_logs/model_3_gru/20230220-181728
Epoch 1/5
215/215 [==============================] - 11s 40ms/step - loss: 0.2190 - accuracy: 0.9086 - val_loss: 0.6836 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1066 - accuracy: 0.9634 - val_loss: 0.7652 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0735 - accuracy: 0.9723 - val_loss: 0.8573 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0607 - accuracy: 0.9742 - val_loss: 1.0761 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0580 - accuracy: 0.9745 - val_loss: 1.2070 - val_accuracy: 0.7677


In [142]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[7.8192890e-02],
       [8.7896377e-01],
       [9.9990082e-01],
       [2.0685147e-01],
       [1.6728419e-04],
       [9.9977785e-01],
       [9.4837773e-01],
       [9.9995065e-01],
       [9.9992287e-01],
       [4.8415953e-01]], dtype=float32)

In [143]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))

In [144]:
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [145]:
calculate_results(model_3_pred,val_labels)

{'accuracy': 76.77165354330708,
 'precision': 0.7712157154143837,
 'recall': 0.7677165354330708,
 'f1': 0.7686398042011251}

### Model-4 Bidirectional RNN

In [146]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1,activation = "sigmoid",)(x)
model_4 = tf.keras.Model(inputs,outputs)

In [147]:
model_4.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 15, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                           

In [148]:
model_4.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [149]:
mode_4_history = model_4.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_4_bidirectional")],
)

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230220-181749
Epoch 1/5
215/215 [==============================] - 20s 57ms/step - loss: 0.1262 - accuracy: 0.9540 - val_loss: 1.1149 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0570 - accuracy: 0.9739 - val_loss: 1.4310 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0486 - accuracy: 0.9783 - val_loss: 1.2612 - val_accuracy: 0.7520
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0425 - accuracy: 0.9797 - val_loss: 1.2951 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0489 - accuracy: 0.9784 - val_loss: 1.3990 - val_accuracy: 0.7454


In [150]:
model_4_preds = tf.squeeze(tf.round(model_4.predict(val_sentences)))

24/24 [==============================] - 5s 10ms/step


In [151]:
calculate_results(val_labels,model_4_preds)

{'accuracy': 74.54068241469817,
 'precision': 0.7461573940792663,
 'recall': 0.7454068241469817,
 'f1': 0.745653698958201}

### Model-5: Conv1D 

In [152]:
embedding_text = embedding(text_vectorizer(['this is a test sentence for embedding']))

In [153]:
embedding_text

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02532874, -0.00020168,  0.01288112, ..., -0.0484078 ,
          0.11103475,  0.01597924],
        [ 0.04781511, -0.01371157, -0.01102706, ..., -0.02351439,
          0.01487272,  0.03745407],
        [ 0.02504591,  0.03204178, -0.02806445, ...,  0.02679012,
         -0.02345852,  0.01356967],
        ...,
        [ 0.01076431, -0.01247481,  0.00437861, ..., -0.00376652,
          0.04662009,  0.00873199],
        [ 0.01076431, -0.01247481,  0.00437861, ..., -0.00376652,
          0.04662009,  0.00873199],
        [ 0.01076431, -0.01247481,  0.00437861, ..., -0.00376652,
          0.04662009,  0.00873199]]], dtype=float32)>

In [154]:
conv_1d = layers.Conv1D(filters = 32,
kernel_size = 5,
activation = "relu",
padding = "same"
)

In [155]:
conv_1d_output = conv_1d(embedding_text)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_text.shape, conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [156]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.04772814, 0.07567579, 0.07567479, 0.05915068, 0.04449635,
        0.04219661, 0.04867159, 0.04718599, 0.06375843, 0.0515985 ,
        0.04731774, 0.05833512, 0.03548823, 0.06808111, 0.03916295,
        0.05062966, 0.05842587, 0.0705367 , 0.04998299, 0.05271178,
        0.07098517, 0.01793418, 0.0226175 , 0.05058339, 0.06398422,
        0.05103598, 0.03771203, 0.0491111 , 0.03276432, 0.03115106,
        0.04432404, 0.05165747]], dtype=float32)>

In [157]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,kernel_size = 5,activation = "relu",padding = "valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name = "model_5_conv1d")

In [158]:
model_5.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [159]:
mode_5_history = model_5.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_5_conv")],
)

Saving TensorBoard log files to: model_logs/model_5_conv/20230220-181915
Epoch 1/5
215/215 [==============================] - 10s 34ms/step - loss: 0.1668 - accuracy: 0.9518 - val_loss: 0.7803 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0809 - accuracy: 0.9731 - val_loss: 0.9330 - val_accuracy: 0.7493
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0650 - accuracy: 0.9771 - val_loss: 0.9992 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0550 - accuracy: 0.9797 - val_loss: 1.0606 - val_accuracy: 0.7507
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0518 - accuracy: 0.9809 - val_loss: 1.1161 - val_accuracy: 0.7520


In [160]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

24/24 [==============================] - 0s 3ms/step


In [161]:
calculate_results(val_labels,model_5_preds)

{'accuracy': 75.19685039370079,
 'precision': 0.7529888699847289,
 'recall': 0.7519685039370079,
 'f1': 0.7497885511234939}